# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,9.86,72,73,8.62,CL,1705712077
1,1,monte alegre,-2.0078,-54.0692,25.92,80,42,1.97,BR,1705712077
2,2,adamstown,-25.0660,-130.1015,24.43,79,40,6.93,PN,1705712077
3,3,ribeira grande,38.5167,-28.7000,14.87,90,97,2.68,PT,1705712077
4,4,whitehorse,60.7161,-135.0538,-25.39,71,20,1.54,CA,1705712003


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

# Configure the map plot
city_humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City",
    alpha=.5
   
)

# Display the map
city_humidity_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
#Criteria:
    #A max temperature lower than 27 degrees but higher than 21
    #Wind speed less than 4.5 m/s
    #Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= 27) & (city_data_df["Max Temp"] >= 21) 
                                & (city_data_df["Wind Speed"] <= 4.5)
                                & (city_data_df["Cloudiness"] == 0)]
# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,margaret river,-33.9500,115.0667,22.51,55,0,0.48,AU,1705712080
52,52,ejura,7.3856,-1.3562,25.79,39,0,2.16,GH,1705712085
81,81,alo,7.4178,6.7371,24.31,18,0,0.24,NG,1705712089
93,93,hermanus,-34.4187,19.2345,22.50,56,0,3.94,ZA,1705712091
144,144,arica,-18.4750,-70.3042,24.75,73,0,1.54,CL,1705711773
254,254,arraial do cabo,-22.9661,-42.0278,24.92,83,0,2.57,BR,1705712112
299,299,ajasse ipo,8.2333,4.8167,21.58,15,0,1.34,NG,1705712119
331,331,kalmunai,7.4167,81.8167,24.98,86,0,1.14,LK,1705712124
388,388,nueve de julio,-35.4444,-60.8831,23.87,43,0,1.34,AR,1705712132
410,410,takoradi,4.8845,-1.7554,26.46,82,0,2.19,GH,1705712136


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Max Temp", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Hotel Name
16,margaret river,AU,-33.9500,115.0667,22.51,55,
52,ejura,GH,7.3856,-1.3562,25.79,39,
81,alo,NG,7.4178,6.7371,24.31,18,
93,hermanus,ZA,-34.4187,19.2345,22.50,56,
144,arica,CL,-18.4750,-70.3042,24.75,73,
254,arraial do cabo,BR,-22.9661,-42.0278,24.92,83,
299,ajasse ipo,NG,8.2333,4.8167,21.58,15,
331,kalmunai,LK,7.4167,81.8167,24.98,86,
388,nueve de julio,AR,-35.4444,-60.8831,23.87,43,
410,takoradi,GH,4.8845,-1.7554,26.46,82,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel
import requests
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat},{lng},{radius}"
    params["bias"] =  f"proximity:{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
margaret river - nearest hotel: No hotel found
ejura - nearest hotel: No hotel found
alo - nearest hotel: No hotel found
hermanus - nearest hotel: No hotel found
arica - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
ajasse ipo - nearest hotel: No hotel found
kalmunai - nearest hotel: No hotel found
nueve de julio - nearest hotel: No hotel found
takoradi - nearest hotel: No hotel found
agboville - nearest hotel: No hotel found
guatrache - nearest hotel: No hotel found
ovalle - nearest hotel: No hotel found
knysna - nearest hotel: No hotel found
santa luzia - nearest hotel: No hotel found


,City,Country,Lat,Lng,Max Temp,Humidity,Hotel Name
16,margaret river,AU,-33.9500,115.0667,22.51,55,No hotel found
52,ejura,GH,7.3856,-1.3562,25.79,39,No hotel found
81,alo,NG,7.4178,6.7371,24.31,18,No hotel found
93,hermanus,ZA,-34.4187,19.2345,22.50,56,No hotel found
144,arica,CL,-18.4750,-70.3042,24.75,73,No hotel found
254,arraial do cabo,BR,-22.9661,-42.0278,24.92,83,No hotel found
299,ajasse ipo,NG,8.2333,4.8167,21.58,15,No hotel found
331,kalmunai,LK,7.4167,81.8167,24.98,86,No hotel found
388,nueve de julio,AR,-35.4444,-60.8831,23.87,43,No hotel found
410,takoradi,GH,4.8845,-1.7554,26.46,82,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
city_humidity_hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City",
    alpha=.5,
    hover_cols=["Hotel Name", "Country"],
    width= 800,
    height=600
)
   

# Display the map
city_humidity_hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)